In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------
# 1. Define error grid (e - ê) over which to evaluate each loss
# ------------------------------------------------------------------
errors = np.linspace(-4, 4, 500)        # prediction error values

# ------------------------------------------------------------------
# 2. Loss functions
# ------------------------------------------------------------------
def mse_loss(e):
    """Mean Squared Error."""
    return e ** 2

def pinball_loss(e, tau):
    """Pinball (quantile) loss for quantile level tau."""
    return np.where(e >= 0, tau * e, (1 - tau) * (-e))

# Evaluate losses
mse_values      = mse_loss(errors)
pinball_tau_01  = pinball_loss(errors, tau=0.1)
pinball_tau_09  = pinball_loss(errors, tau=0.9)

# ------------------------------------------------------------------
# 3. Plotting
# ------------------------------------------------------------------
plt.figure(figsize=(8, 6))

# Plot MSE
plt.plot(errors, mse_values, label="MSE", color="black", linewidth=2)

# Plot pinball losses
plt.plot(errors, pinball_tau_01,
         label=r"Pinball $(\tau=0.1)$", color="royalblue", linewidth=2)
plt.plot(errors, pinball_tau_09,
         label=r"Pinball $(\tau=0.9)$", color="darkorange", linewidth=2)

# Aesthetics
plt.title("MSE vs. Pinball Loss")
plt.xlabel(r"Prediction error $(e - \hat{e})$")
plt.ylabel("Loss value")
plt.axvline(0, color="gray", linestyle="--", linewidth=1)  # zero-error marker
plt.ylim(bottom=0)
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()

# ------------------------------------------------------------------
# 4. Display
# ------------------------------------------------------------------
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))

# Plot MSE
plt.plot(errors, pinball_tau_01,
         label=r"Pinball $(\tau=0.1)$", color="royalblue", linewidth=2)

# Aesthetics
plt.title("Pinball Loss ($\tau=0.1$)")
plt.xlabel(r"Prediction error $(e - \hat{e})$")
plt.ylabel("Loss value")
plt.axvline(0, color="gray", linestyle="--", linewidth=1)  # zero-error marker
plt.ylim(bottom=0)
plt.grid(alpha=0.3)
#plt.legend()
plt.tight_layout()

# ------------------------------------------------------------------
# 4. Display
# ------------------------------------------------------------------
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

# Plot MSE
plt.plot(errors, mse_values, label="MSE", color="black", linewidth=2)

# Aesthetics
plt.title("MSE")
plt.xlabel(r"Prediction error $(e - \hat{e})$")
plt.ylabel("Loss value")
plt.axvline(0, color="gray", linestyle="--", linewidth=1)  # zero-error marker
plt.ylim(bottom=0)
plt.grid(alpha=0.3)
#plt.legend()
plt.tight_layout()

# ------------------------------------------------------------------
# 4. Display
# ------------------------------------------------------------------
plt.show()

Results refactor

In [ ]:
import pandas as pd

read_file = "results_no_plot.csv"
df = pd.read_csv(read_file)

cl_cost_col = "Closed Loop Cost"

# remove the squre brackets from the values in the column
df[cl_cost_col] = df[cl_cost_col].str.replace(r'\[|\]', '', regex=True).astype(float)

In [ ]:
df[cl_cost_col]

# display column names whose values are numeric
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
print("Numeric columns:", numeric_columns)

In [ ]:
df.to_csv("results_no_plot_cleaned.csv", index=False)

In [2]:
import pandas as pd
df = pd.read_csv("results_no_plot_cleaned.csv")

In [3]:
df.columns

Index(['Serial No.', 'Date', 'Type', 'No Samples', 'Surrogate Nodes',
       'CQR Nodes', 'CQR Alpha', 'Initial Condition', 'Optimizer Success',
       'Closed Loop Cost', 'Concentration Cb',
       'Boundary Violation Probability', 'Iterations', 'MPC Horizon',
       'MPC Input Cost', 'ICBMPC Tightner', 'ICBMPC Random Sampling',
       'ICBMPC Confidence Cutoff', 'MPC Setpoint', 'ICBMPC Max Search', 'Time',
       'LQR r', 'LQR q', 'MPC Robust Horizon'],
      dtype='object')

In [9]:
def calculate_average(df, column_name):
    
    unique_values = df[column_name].unique()
    type_col ="Type"
    cases = df[type_col].unique()

    df_new = pd.DataFrame(columns=[column_name, "Type", 'Avg Optimizer Success', 'Avg Closed Loop Cost', 'Avg Concentration Cb', 'Avg Boundary Violation Probability', 'Avg Time'])

    performance_metric_col = ['Optimizer Success', 'Closed Loop Cost', 'Concentration Cb', 'Boundary Violation Probability', 'Time']

    for value in unique_values:
        for case in cases:
            df_filtered = df[(df[column_name] == value) & (df[type_col] == case)]
            if not df_filtered.empty:
                avg_values = {
                    column_name: value,
                    'Type': case,
                    'Avg Optimizer Success': df_filtered['Optimizer Success'].mean(),
                    'Avg Closed Loop Cost': df_filtered['Closed Loop Cost'].mean(),
                    'Avg Concentration Cb': df_filtered['Concentration Cb'].mean(),
                    'Avg Boundary Violation Probability': df_filtered['Boundary Violation Probability'].mean(),
                    'Avg Time': df_filtered['Time'].mean()
                }

                # convert to DataFrame and append
                df_avg = pd.DataFrame([avg_values])
                df_new = pd.concat([df_new, df_avg], ignore_index=True)
    
    # save the new DataFrame to a CSV file
    df_new.to_csv(f"Average_{column_name}.csv", index=False)


    return df_new


In [15]:
df_new = calculate_average(df, "LQR r")

/var/folders/bt/607j6q3j63vd7szlx_ssvyg40000gn/T/ipykernel_3032/3679582460.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_avg], ignore_index=True)


In [14]:
df_new

,ICBMPC Max Search,Type,Avg Optimizer Success,Avg Closed Loop Cost,Avg Concentration Cb,Avg Boundary Violation Probability,Avg Time
0,3.0,cs1,0.075833,122.786392,0.852520,0.005353,17.626764
1,3.0,cs5,0.084167,130.333887,0.846285,0.005255,8.384151
2,5.0,cs1,0.197143,98.803123,0.747668,0.018630,26.045947
3,5.0,cs5,0.195510,105.016557,0.750626,0.018253,12.354329
4,10.0,cs1,0.143000,152.795370,0.924286,0.007810,52.485356
5,10.0,cs5,0.150500,140.470710,0.922113,0.007941,25.865486
